In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [111]:
search_df = pd.read_csv('csv/search.csv', index_col='id')
search_df.head()

# define the text fields to search over and their corresponding weights
text_fields = ['name', 'author', 'series', 'shortDesc_clean_noStop', 'longDesc_clean_noStop', 'shortDesc_clean', 'longDesc_clean']
weights = [30, 50, 10, 5, 5, 5]  # adjust weights as desired

# create a CountVectorizer instance to tokenize the text data
vectorizer = CountVectorizer(stop_words='english')

# fit the vectorizer to the text data and transform the text fields into bag-of-words representations
text_vectors = vectorizer.fit_transform(search_df[text_fields].apply(lambda x: ' '.join(x.dropna()), axis=1))
def search(query, n=10, threshold=0.01):
    # convert the query string to a bag-of-words representation
    query_vector = vectorizer.transform([query])

    # compute the cosine similarity between the query vector and the document vectors
    similarities = cosine_similarity(query_vector, text_vectors)

    # compute the weighted average of the cosine similarities for each document
    # weighted_similarities = np.average(similarities, axis=0, weights=weights)

    # filter out documents with similarity scores below the threshold
    above_threshold = similarities >= threshold
    if not np.any(above_threshold):
        return pd.DataFrame()  # return an empty dataframe if no documents are above the threshold
    else:
        top_indices = np.argsort(similarities)[0][::-1][:n]
        return search_df[['name', 'author', 'series', 'shortDescription']].iloc[top_indices]
        
    
search('Heliceo', 5)

,name,author,series,shortDescription
id,,,,
24,The Prophecy,Heliceo,Ragecraft,"Are you looking for a huge, challenging, heavi..."
21,The Corrupted Path,Heliceo,Ragecraft,"Are you looking for a huge, challenging, heavi..."
23,Insomnia,Heliceo,Ragecraft,"Are you looking for a huge, challenging, heavi..."
25,Kitten's Revolt,The Cool Cats Cuteyard,Kitten's Revolt Series,Originally designed to be built quickly as a c...
9,WinterStorm,Tecnocraft2802,Seasons Time,WINTER STORM es un mapa MiniCTM de la 1.12.2.<...
